<a href="https://colab.research.google.com/github/yueop/AS_LAB/blob/main/eval_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from google.colab import drive

# 1. 드라이브 마운트
drive.mount('/content/drive')

# 2. 경로 설정
PROJECT_PATH = '/content/drive/MyDrive/AS_LAB'
if not os.path.exists(PROJECT_PATH):
    os.makedirs(PROJECT_PATH)

# 3. 시스템 경로 추가 및 작업 디렉토리 변경
if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)
os.chdir(PROJECT_PATH)

print(f"현재 위치: {os.getcwd()}")

Mounted at /content/drive
현재 위치: /content/drive/MyDrive/AS_LAB


In [ ]:
# 파일 이름을 명시적으로 지정
FILE_NAME = "eval.py"
# 전체 경로 사용
SAVE_PATH = f"/content/drive/MyDrive/AS_LAB/{FILE_NAME}"

In [ ]:
content = """
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import yaml
import random
from sklearn.metrics import confusion_matrix, accuracy_score
import os

from my_datasets import UnifiedDataLoader
from models import get_model

#config 파일 읽어오는 함수
def load_config(config_path):
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    return config

#시드 고정 함수
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

#오분류 이미지 시각화 함수
def visualize_misclassified(inputs, labels, preds, dataset_name, save_dir, num_images=10):
    #IRIS는 이미지 데이터가 아니므로 생략
    if dataset_name == 'iris':
        return

    #저장할 폴더가 없다면 생성
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    plt.figure(figsize=(12, 6))
    limit = min(len(labels), num_images)

    for i in range(limit):
        plt.subplot(2, 5, i + 1)
        #Tensor(C, H, W) -> NumPy(H, W) 변환(흑백)
        img = inputs[i].cpu().squeeze() #squeeze: 텐서의 차원 중 크기가 1인 차원을 제거하는 함수.
        img = img * 0.5 + 0.5 #정규화 해제(-1~1 -> 0~1)
        img = np.clip(img, 0, 1)

        plt.imshow(img, cmap='gray')
        plt.title(f"True:{labels[i]} | Pred:{preds[i]}", fontsize=10, color='red')
        plt.axis('off')

    plt.tight_layout()

    save_path = os.path.join(save_dir, 'misclassified_sample.png')
    plt.savefig(save_path)
    print(f"오분류 이미지 저장 완료: {save_path}")
    plt.close()

#메인 함수(평가 루프)
def main():
    #1. 설정 로드(config, 시드, 장치)
    cfg = load_config('config.yaml')
    set_seed(cfg['train']['seed'])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    base_dir = os.getcwd()
    result_dir = os.path.join(base_dir, 'results', cfg['project_name'])
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    print(f"평가 시작. Project: {cfg['project_name']}")

    #2. 데이터 로더
    data_loader = UnifiedDataLoader(cfg)
    test_loader = data_loader.get_loader(cfg['data']['dataset_name'], 'test')

    #3. 모델 초기화 및 가중치 로드
    dropout_rate = cfg['model'].get('dropout_rate', 0.0)

    #모델 뼈대 생성
    model = get_model(
        cfg['model']['type'],
        cfg['model']['input_size'],
        cfg['model']['hidden_size'],
        cfg['model']['num_classes'],
        dropout_rate = dropout_rate
    )

    #저장된 가중치 불러오기(핵심 단계)
    model_path = cfg['train']['save_path']
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location=device))
        print(f"모델 가중치 로드 성공: {model_path}")
    else:
        print(f"저장된 모델 파일이 없습니다: {model_path}")
        return

    model = model.to(device)
    model.eval() #평가 모드로 전환

    #4. 추론
    all_preds = []
    all_labels = []

    #오분류 데이터 수집용 리스트
    mis_inputs = []
    mis_labels_list = []
    mis_preds_list = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            #결과 모으기(gpu 텐서를 cpu에 옮겨서 리스트에 추가)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            #오분류 이미지 수집
            if len(mis_inputs) < 10:
                wrong_idx = (predicted != labels).nonzero(as_tuple=True)[0] #예측이 틀린 인덱스를 찾아 튜플로 반환 후 리스트로 꺼냄
                if len(wrong_idx) > 0:
                    mis_inputs.extend(inputs[wrong_idx].cpu()) #배치 전체에서 틀린 이미지들 추출
                    mis_labels_list.extend(labels[wrong_idx].cpu().tolist()) #정답
                    mis_preds_list.extend(predicted[wrong_idx].cpu().tolist()) #예측값

    #5. 성능 지표 계산 및 시각화
    #5-1 정확도 출력
    acc = accuracy_score(all_labels, all_preds)
    print(f"최종 테스트 정확도: {acc*100:.2f}%")

    #5-2 Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix - {cfg["project_name"]}')

    cm_path = os.path.join(result_dir, 'confusion_matrix.png')
    plt.savefig(cm_path)
    print(f"Confusion Matrix 저장 완료: {cm_path}")
    plt.close()

    #5-3 오분류 이미지 출력
    if cfg['data']['dataset_name'] != 'iris' and len(mis_inputs) > 0:
        #리스트에 담긴 텐서들 다시 하나의 배치로 합침
        mis_inputs = torch.stack(mis_inputs)[:10]
        mis_labels_list = mis_labels_list[:10]
        mis_preds_list = mis_preds_list[:10]

        visualize_misclassified(mis_inputs, mis_labels_list, mis_preds_list, cfg['data']['dataset_name'], save_dir = result_dir)

if __name__ == "__main__":
    main()
"""

try:
    with open(SAVE_PATH, 'w', encoding='utf-8') as f:
        f.write(content.strip())
    print(f"파일이 안전하게 생성되었습니다: {SAVE_PATH}")
except Exception as e:
    print(f"저장 실패: {e}")

파일이 안전하게 생성되었습니다: /content/drive/MyDrive/AS_LAB/eval.py
